## Triangle Similarity for Object/Marker to Camera Distance

In order to determine the distance from our camera to a known object or marker, we are going to utilize triangle similarity.

Given a marker or object with a known width W.<br/>
When this marker is some distance D from our camera.<br>
And a picture of our object is taken from a camera is analysed to obtain the apparent width in pixels P. <br/>
Then we can derive the perceived focal length F of our camera<br/>

`F = (P x  D) / W`<br/>

Place a standard piece of 8.5 x 11in piece of paper (horizontally; W = 11) D = 24 inches in front of a camera and capture the image. <br/>
Measure the width of the piece of paper in the image <br/>
Let the perceived width of the paper be P = 248 pixels. <br/>

<br/>let focal length F be calculated:

`F = (248px x 24in) / 11in = 543.45`

Continue to move the camera both closer and farther away from the object/marker, and capply the triangle similarity to determine the distance of the object to the camera:

`D’ = (W x F) / P`

Let the camera position be 3 ft (or 36 inches) away from the marker.<br/>
Let the perceived width of the piece of the marker be 170 pixels. 
Let Distance be calculated:

`D’ = (11in x 543.45) / 170 = 35in`

Or roughly 36 inches, which is 3 feet.

In [1]:
# import the necessary packages
from imutils import paths
import numpy as np
import imutils
import cv2

def find_marker(image):
    # convert the image to grayscale, blur it, and detect edges
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 35, 125)
    # find the contours in the edged image and keep the largest one;
    # we'll assume that this is our piece of paper in the image
    cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key = cv2.contourArea)
    # compute the bounding box of the of the paper region and return it
    return cv2.minAreaRect(c)

In [2]:
# This function takes a knownWidth of the marker, a computed focalLength , and perceived width of an object in an 
# image (measured in pixels), and applies the triangle similarity detailed above to compute the actual distance to
# the object.

def distance_to_camera(knownWidth, focalLength, perWidth):
    # compute and return the distance from the maker to the camera
    return (knownWidth * focalLength) / perWidth

Use a single image to calibrate with

In [3]:
# initialize the known distance from the camera to the object, which
# in this case is 24 inches
KNOWN_DISTANCE = 24.0
# initialize the known object width, which in this case, the piece of
# paper is 12 inches wide
KNOWN_WIDTH = 11.0
# load the image that contains an object that is KNOWN TO BE 2 feet
# from our camera, then find the paper marker in the image, and initialize
# the focal length
image = cv2.imread("images/2ft.png")
marker = find_marker(image)
focalLength = (marker[1][0] * KNOWN_DISTANCE) / KNOWN_WIDTH

Now that we have a calibration then we can make judgements about other images based on this calibration

In [4]:
# loop over all the test images
for imagePath in sorted(paths.list_images("images")):
    # load the image, find the marker in the image, then compute the
    # distance to the marker from the camera
    image = cv2.imread(imagePath)
    marker = find_marker(image)
    inches = distance_to_camera(KNOWN_WIDTH, focalLength, marker[1][0])
    # draw a bounding box around the image and display it
    box = cv2.cv.BoxPoints(marker) if imutils.is_cv2() else cv2.boxPoints(marker)
    box = np.int0(box)
    cv2.drawContours(image, [box], -1, (0, 255, 0), 2)
    cv2.putText(image, "%.2fft" % (inches / 12),
        (image.shape[1] - 200, image.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX,
        2.0, (0, 255, 0), 3)
    cv2.imshow("image", image)
    cv2.waitKey(0)

cv2.destroyAllWindows()